Produce the database by fusing the PhageScope databases

In [ ]:
from google.colab import drive
import pandas as pd
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Specify the folder path (ensure this is correct)
folder_path = '/content/drive/MyDrive/databasefagi/'  # Replace 'FolderName' with the actual folder name

# List all files in the directory
files = os.listdir(folder_path)
print("Files in the directory:")
print(files)

# Step 3: Use glob to find all .tsv files in the folder
# You can also check the specific files to confirm their extension
tsv_files = [f for f in files if f.endswith('.tsv')]

if not tsv_files:
    raise ValueError("No .tsv files found in the specified folder.")

# Initialize an empty list to hold the DataFrames
dataframes = []

# Read each file into a DataFrame and append to the list
for file in tsv_files:
    df = pd.read_csv(file, sep='\t')  # Use sep='\t' for TSV files
    dataframes.append(df)

# Concatenate all DataFrames in the list
combined_data = pd.concat(dataframes, ignore_index=True)  # Reset index after concatenation
.
# Check the structure of the combined DataFrame
print(combined_data.head())
print(combined_data.info())

# Save the combined DataFrame if needed
combined_data.to_csv('/content/drive/MyDrive/integrated_phagescope_data.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in the directory:
['refseq_phage_meta_data.tsv', 'genbank_phage_meta_data.tsv', 'embl_phage_meta_data.tsv', 'ddbj_phage_meta_data.tsv', 'phagesdb_phage_meta_data.tsv', 'gvd_phage_meta_data.tsv', 'gpd_phage_meta_data.tsv', 'temphd_phage_meta_data.tsv', 'mgv_phage_meta_data.tsv', 'igvd_phage_meta_data.tsv', 'chvd_phage_meta_data.tsv', 'gov2_phage_meta_data.tsv', 'stv_phage_meta_data.tsv', 'img_vr_phage_meta_data.tsv']
      Phage_ID  Length  GC_content      Taxonomy    Completeness  \
0  NC_001330.1    6087   45.178249  Microviridae    High-quality   
1  NC_001331.1    7349   61.491359    Inoviridae     Low-quality   
2  NC_001332.1    6744   42.719454    Inoviridae  Medium-quality   
3  NC_001335.1   52297   62.257873  Caudovirales    High-quality   
4  NC_001341.1    4491   33.288800           NaN  Not-determined   

                      Host  Lifestyl

Produce the host list from Phagescope database

In [ ]:
import pandas as pd

# Load the integrated phage scope data file
combined_data_path = '/content/drive/MyDrive/integrated_phagescope_data.csv'
combined_data = pd.read_csv(combined_data_path)

# Check if the 'Host' column exists
if 'Host' not in combined_data.columns:
    raise ValueError("The 'Host' column is not found in the integrated data.")

# Split the 'Host' column by commas or semicolons, and create a single series of hosts
hosts_expanded = combined_data['Host'].str.split(',|;', expand=True).stack().reset_index(drop=True)

# Strip whitespace and filter out any empty entries
hosts_expanded = hosts_expanded.str.strip()
hosts_expanded = hosts_expanded[hosts_expanded != '']  # Remove empty strings

# Get unique hosts, ensuring no duplicates remain
unique_hosts = pd.Series(hosts_expanded).unique()

# Convert to a sorted list for better readability
unique_hosts_list = sorted(unique_hosts)

# Display the unique hosts
print("Unique Hosts:")
for host in unique_hosts_list:
    print(host)

# Optional: Save the unique hosts to a file
unique_hosts_df = pd.DataFrame(unique_hosts_list, columns=['Unique_Hosts'])
unique_hosts_df.to_csv('/content/drive/MyDrive/unique_hosts.csv', index=False)


<ipython-input-11-a9a6dd172f9e>:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data = pd.read_csv(combined_data_path)


Streaming output truncated to the last 5000 lines.
Mucilaginibacter mali
Mucinivorans
Mucispirillum schaedleri
Murdochiella
Murdochiella massiliensis
Muribaculaceae
Muribaculaceae bacterium
Muribaculum
Muribaculum gordoncarteri
Muribaculum intestinale
Muribaculum sp. NM65 B17
Muribaculum sp001701195
Muricauda sp000745185
Muricauda sp004804315
Muricauda sp013407935
Muricomes
Muricomes contortus B
Muricomes contortus C
Muricomes fissicatena A
Muricomes intestini
Muricomes oroticus
Muriiphilus sp900177545
Murimonas
Mus musculus
Mycetocola A miduiensis
Mycetocola B reblochoni
Mycetocola miduiensis
Mycetohabitans
Mycetohabitans rhizoxinica
Mycetohabitans sp900155955
Mycoavidus cysteinexigens
Mycobacteriaceae
Mycobacteriales
Mycobacterium
Mycobacterium abscessus
Mycobacterium aichiense
Mycobacterium alsense
Mycobacterium aquaticum
Mycobacterium arosiense A
Mycobacterium arupense
Mycobacterium asiaticum
Mycobacterium avium
Mycobacterium brisbanense
Mycobacterium canariasense
Mycobacterium che

Produce a reduced database for proteins specifically involved in infection (viral receptor)

In [ ]:
import pandas as pd
import glob

# Specify the folder path where the .tsv files are located
folder_path = '/content/drive/MyDrive/databaseproteine/'

# Load all .tsv files in the specified folder
all_files = glob.glob(folder_path + "*.tsv")

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each file and read it
for file in all_files:
    df = pd.read_csv(file, sep='\t', usecols=['Phage_ID', 'Protein_ID', 'Product', 'Protein_classification'])
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Filter rows where 'Protein_classification' contains 'infection' (case insensitive)
filtered_df = combined_df[combined_df['Protein_classification'].str.contains('infection', case=False, na=False)]

# Save the filtered DataFrame to a new .tsv file
output_file_path = '/content/drive/MyDrive/filtered_phage_data.tsv'  # Change this to your desired output path
filtered_df.to_csv(output_file_path, sep='\t', index=False)

print(f"Filtered data saved to {output_file_path}")


Filtered data saved to /content/drive/MyDrive/filtered_phage_data.tsv


**THE FOLLOWING STEPS ARE THE ACTUAL PROGRAM**



Perform exact search in the database

In [ ]:
!pip install fuzzywuzzy
from google.colab import drive
import pandas as pd
from fuzzywuzzy import process
drive.mount('/content/drive')

# Load the integrated phage scope data file
combined_data_path = '/content/drive/MyDrive/integrated_phagescope_data.csv'
combined_data = pd.read_csv(combined_data_path)

# Check if the 'Host' column exists
if 'Host' not in combined_data.columns:
    raise ValueError("The 'Host' column is not found in the integrated data.")

# Split the 'Host' column by commas or semicolons, and create a single series of hosts
hosts_expanded = combined_data['Host'].str.split(',|;', expand=True).stack().reset_index(drop=True)

# Strip whitespace and filter out any empty entries
hosts_expanded = hosts_expanded.str.strip()
hosts_expanded = hosts_expanded[hosts_expanded != '']  # Remove empty strings

# Get unique hosts
unique_hosts = pd.Series(hosts_expanded).unique()
unique_hosts_list = sorted(unique_hosts)

# Function to search for a host with fuzzy matching
def search_host(target_host):
    match = process.extractOne(target_host, unique_hosts_list)
    return match

# Ask user for the target host name
target_host = input("Enter the target host name: ")

# Search for the target host in the unique hosts list
result = search_host(target_host)

# Check if a match was found
if result and result[1] >= 80:  # Confidence threshold
    print(f"Match found: {result[0]} (Confidence: {result[1]}%)")

    # Retrieve all phage IDs matching the found host
    matching_host = result[0]
    # Filter the original dataframe for the matching host
    combined_data_cleaned = combined_data.dropna(subset=['Host'])  # Drop rows where 'Host' is NaN
    matching_phages = combined_data_cleaned[combined_data_cleaned['Host'].str.contains(matching_host, case=False)]

    # Extract the Phage_IDs
    phage_ids = matching_phages['Phage_ID'].tolist()

    if phage_ids:
        print("Phage IDs matching the host:")
        for phage_id in phage_ids:
            print(phage_id)
    else:
        print("No phage IDs found for this host.")
else:
    print("No close match found for the specified host. Proceeding with search of relatives in the host list.")
    !pip install biopython
    import requests
    from Bio import Entrez
    from fuzzywuzzy import process
    # Function to get taxonomy details
    def get_taxonomy_info(taxid):
        if not isinstance(taxid, str) or not taxid.isdigit():
            print(f"Invalid taxid: {taxid}")
            return None
        try:
            fetch_handle = Entrez.efetch(db="taxonomy", id=taxid, retmode="xml")
            fetch_results = Entrez.read(fetch_handle)
            fetch_handle.close()

            if not fetch_results:
                print(f"No taxonomy information found for TaxID: {taxid}")
                return None

            lineage = fetch_results[0].get('Lineage', '').split("; ")
            scientific_name = fetch_results[0].get('ScientificName', None)

            genus = lineage[-2] if len(lineage) >= 2 else None
            family = lineage[-3] if len(lineage) >= 3 else None

            return {
                "lineage": lineage,
                "genus": genus,
                "family": family,
                "scientific_name": scientific_name
            }

        except Exception as e:
            print(f"Error fetching taxonomy information for TaxID: {taxid}, Error: {e}")
            return None

    # Function to find species in the same genus or family using NCBI Entrez
    def find_species_in_taxonomy(taxid, rank):
        search_handle = Entrez.esearch(db="taxonomy", term=f"txid{taxid}[Subtree] AND {rank}[Rank]", retmax=100)
        search_results = Entrez.read(search_handle)
        search_handle.close()

        if not search_results["IdList"]:
            print(f"No {rank}-level relatives found for TaxID: {taxid}")
            return []

        species_list = []
        for species_id in search_results["IdList"]:
            species_info = get_taxonomy_info(species_id)
            if species_info and "scientific_name" in species_info and species_info["scientific_name"]:
                species_list.append(species_info["scientific_name"])

        return species_list

    def get_taxid(species_name):
        try:
            Entrez.email = "your_email@example.com"  # Always provide your email for NCBI
            search_handle = Entrez.esearch(db="taxonomy", term=species_name)
            search_results = Entrez.read(search_handle)
            search_handle.close()
            if search_results["IdList"]:
                return search_results["IdList"][0]  # Return the first match's taxid
            else:
                print(f"No TaxID found for species: {species_name}")
                return None
        except Exception as e:
            print(f"Error retrieving TaxID for {species_name}: {e}")
            return None

    # Function to find the closest relatives based on taxonomy information
    def find_closest_relatives(taxid, host_list):
        taxonomy_info = get_taxonomy_info(taxid)
        if taxonomy_info is None:
            print(f"No taxonomy information available for TaxID {taxid}.")
            return []

        genus_species = find_species_in_taxonomy(taxid, 'species')
        matches = find_best_fuzzy_matches(genus_species, host_list, confidence_threshold=94)

        if matches:
            return matches

        family_species = []
        if taxonomy_info['family']:
            family_taxid = get_taxid(taxonomy_info['family'])  # Assuming get_taxid is defined elsewhere
            if family_taxid:
                family_species = find_species_in_taxonomy(family_taxid, 'species')

        return find_best_fuzzy_matches(family_species, host_list, confidence_threshold=94)

    # Function to find the best fuzzy matches with a confidence threshold
    def find_best_fuzzy_matches(species_list, host_list, confidence_threshold):
        matches_dict = {}

        for species_name in species_list:
            best_match = process.extractOne(species_name, host_list)
            if best_match and best_match[1] >= confidence_threshold:
                match_name = best_match[0]
                if match_name not in matches_dict:
                    matches_dict[match_name] = []
                matches_dict[match_name].append(species_name)

        # Prepare the output list
        output = []
        for match, species_names in matches_dict.items():
            species_group = ", ".join(species_names)
            output.append(f"Species: {species_group} | Best Match: {match}")

        return output

    # Example usage
    taxid = input("Enter the target species Taxonomy ID on NCBI Taxonomy browser: ")
    matching_relatives = find_closest_relatives(taxid, unique_hosts_list)

    print("Matching relatives in the host list:")
    for relative in matching_relatives:
        print(relative)
    target_host = input("Try the phage search entering one of the relatives name:")
    result = search_host(target_host)
    # Check if a match was found
    if result and result[1] >= 80:  # Confidence threshold
        print(f"Match found: {result[0]} (Confidence: {result[1]}%)")

        # Retrieve all phage IDs matching the found host
        matching_host = result[0]
        # Filter the original dataframe for the matching host
        combined_data_cleaned = combined_data.dropna(subset=['Host'])  # Drop rows where 'Host' is NaN
        matching_phages = combined_data_cleaned[combined_data_cleaned['Host'].str.contains(matching_host, case=False)]

        # Extract the Phage_IDs
        phage_ids = matching_phages['Phage_ID'].tolist()

        if phage_ids:
            print("Phage IDs matching the host:")
            for phage_id in phage_ids:
                print(phage_id)
        else:
            print("No phage IDs found for this host.")
    else:
        print("No close match found for the specified host. Proceeding with search of relatives in the host list.")



/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Mounted at /content/drive


<ipython-input-5-e33bd778c136>:9: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data = pd.read_csv(combined_data_path)


Streaming output truncated to the last 5000 lines.
IMGVR_UViG_2778261574_000005|2778261574|2778324797|66940-100483
IMGVR_UViG_2778261575_000002|2778261575|2778324896|2-35875
IMGVR_UViG_2778261575_000005|2778261575|2778324920|2-38959
IMGVR_UViG_2778261575_000007|2778261575|2778324927|28049-47897
IMGVR_UViG_2778261576_000004|2778261576|2778325001|55269-96640
IMGVR_UViG_2778261576_000007|2778261576|2778325024|365-33267
IMGVR_UViG_2778261578_000003|2778261578|2778325338|138508-173311
IMGVR_UViG_2778261578_000004|2778261578|2778325340|179336-224159
IMGVR_UViG_2778261578_000006|2778261578|2778325354|24453-64973
IMGVR_UViG_2778261579_000010|2778261579|2778325414|75631-116709
IMGVR_UViG_2778261588_000002|2778261588|2778326186|82076-144164
IMGVR_UViG_2778261588_000003|2778261588|2778326188|21315-51621
IMGVR_UViG_2675903505_000011|2675903505|2675982384|6969-22985
IMGVR_UViG_2675903505_000012|2675903505|2675982394|36178-98391
IMGVR_UViG_2675903536_000011|2675903536|2675986548|129-37239
IMGVR_UViG

Once you found the phages, get the proteins.

In [ ]:
filtered_df = pd.read_csv('/content/drive/MyDrive/filtered_phage_data.tsv', sep='\t')

# Filter the DataFrame to get only the rows where the Phage_ID matches one of the phage_ids
matching_proteins = filtered_df[filtered_df['Phage_ID'].isin(phage_ids)]

# Group the matching proteins by 'Phage_ID'
grouped_proteins = matching_proteins.groupby('Phage_ID')

# Create an output dictionary to store proteins grouped by Phage_ID
output_dict = {phage_id: group[['Protein_ID', 'Product']].to_dict('records') for phage_id, group in grouped_proteins}

# Print the grouped proteins
for phage_id, proteins in output_dict.items():
    print(f"Phage ID: {phage_id}")
    for protein in proteins:
        print(f"  Protein ID: {protein['Protein_ID']}, Product: {protein['Product']}")
    print("\n")

Streaming output truncated to the last 5000 lines.
  Protein ID: uvig_541380_58, Product: tail fiber assembly protein


Phage ID: uvig_541420
  Protein ID: uvig_541420_6, Product: Ead/Ea22-like protein
  Protein ID: uvig_541420_7, Product: Ead/Ea22-like protein
  Protein ID: uvig_541420_16, Product: Ead/Ea22-like protein
  Protein ID: uvig_541420_20, Product: superinfection exclusion SieA-like
  Protein ID: uvig_541420_30, Product: Ead/Ea22-like protein
  Protein ID: uvig_541420_52, Product: P22 tail accessory factor
  Protein ID: uvig_541420_54, Product: tail needle knob
  Protein ID: uvig_541420_59, Product: superinfection exclusion SieA-like


Phage ID: uvig_541632
  Protein ID: uvig_541632_28, Product: Phage portal protein
  Protein ID: uvig_541632_31, Product: Phage gp6-like head-tail connector protein
  Protein ID: uvig_541632_32, Product: Phage head-tail joining protein
  Protein ID: uvig_541632_33, Product: tail completion or Neck1 protein
  Protein ID: uvig_541632_34, Product:

get the protein sequences for the found proteins

In [ ]:
import pandas as pd

# Load the CSV file (replace 'your_file.csv' with the actual path to your CSV file)
csv_file_path = '/content/drive/MyDrive/filtered_merged_output.csv'
df = pd.read_csv(csv_file_path, header=None)

# Extract the list of Protein_IDs from output_dict
protein_ids = {entry['Protein_ID'] for key in output_dict for entry in output_dict[key]}

# Filter the DataFrame to keep only rows where the first column matches one of the Protein_IDs
filtered_df = df[df[0].isin(protein_ids)]

# Extract the second column as a list
second_column_values = filtered_df[1].tolist()

# The result is stored in `second_column_values`
second_column_values

<ipython-input-7-ae845f562ebf>:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path, header=None)


['...',
 'MADYGLTDAGFVIPTFDDILNNYMTALKNTFWCRYSYI',
 '...',
 'LRKAFEKPDNADDIYKTIIEGYKT',
 '...',
 ...]

now I want a script that, given a GenBank accession number for the genome of the target, connects to PHASTEST to get the sequences of the phage genes

In [ ]:
import requests
import time

# Prompt user for accession number
accession_number = input("Enter the GenBank accession number: ")

# URL to access PHASTEST job details
url = f"https://phastest.ca/jobs/{accession_number}/detail.txt"

try:
    # Send GET request to the URL
    response = requests.get(url)

    # Wait for a short time if the server needs a delay
    time.sleep(3)  # Adjust sleep time if necessary

    # Check if the request was successful
    if response.status_code == 200:
        # Get the text response
        data = response.text

        # Keywords to search for
        keywords = [
            "tail protein",
            "phage-like protein",
            "portal protein",
            "hypothetical protein",
            "head protein",
            "plate protein",
            "fiber protein",
            "membrane protein"
        ]

        # List to store protein sequences
        protein_sequences = []

        # Extract lines containing the keywords
        for line in data.splitlines():
            if any(keyword in line.lower() for keyword in keywords):
                print(line)

                # Find and extract the protein sequence (assuming it's at the end of each line)
                sequence = line.split()[-1]

                # Add sequence to protein_sequences list
                protein_sequences.append(sequence)

        # Print all collected sequences for verification
        print("\nCollected Protein Sequences:")
        for seq in protein_sequences:
            print(seq)

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {e}")


Enter the GenBank accession number: NZ_AP014565.1
complement(370630..370755)         PHAGE_Salmon_118970_sal3_NC_031940: hypothetical protein; PP_00326; phage            1.00e-20            -                   NONE                               NONE                               NONE                               MTTITKERIELYVKSPLENGLTRGEQMDLARIALASLEAEPI
complement(371683..371982)         PHAGE_Escher_HK75_NC_016160: hypothetical protein; PP_00329; phage                   2.98e-65            -                   NONE                               NONE                               NONE                               MSVCLIDKRRRGQQIPSVEMPNHTWFCVLDIDGMDTLIDTRHYCDTATATPAKAKKMAALIENWTPPDGWCNGNDRDWHEKMKGYICDFLRKCNGFMVM
complement(371979..372377)         PHAGE_Escher_ECP1_NC_049926: hypothetical protein; PP_00330; phage                   2.58e-39            -                   NONE                               NONE                               NONE                               MKQMTLIEMDGF

In [ ]:
Look for sequence identities between the infection proteins of the prophages and those of the phages of close relatives of the target

In [ ]:
!pip install biopython
from Bio import pairwise2
from Bio.Seq import Seq
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Define a similarity threshold (e.g., 70%)
similarity_threshold = 0.7

# Store significant homologies
significant_homologies = []

# Function to process one pair of sequences
def process_alignment(input_seq, output_seq, input_seq_id, output_seq_id):
    output_seq = Seq(output_seq)
    alignments = pairwise2.align.localxx(input_seq, output_seq)
    for alignment in alignments:
        similarity = alignment[2] / alignment[4]
        if similarity >= similarity_threshold:
            return {
                'Input_Sequence_ID': input_seq_id,
                'Output_Protein_ID': output_seq_id,
                'Output_Product': "N/A",  # Placeholder
                'Similarity': similarity,
                'Alignment': alignment
            }
    return None

# Total counts for tracking progress
total_proteins = len(protein_sequences)
total_outputs = len(second_column_values)

print(f"Starting alignment of {total_proteins} protein sequences against {total_outputs} output sequences.")
start_time = time.time()

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor() as executor:
    futures = []
    for i, input_seq in enumerate(protein_sequences):
        print(f"\nSubmitting tasks for input sequence {i+1}/{total_proteins}")

        for j, output_seq in enumerate(second_column_values):
            futures.append(executor.submit(process_alignment, input_seq, output_seq, i, j))

    for k, future in enumerate(as_completed(futures)):
        result = future.result()
        if result:
            significant_homologies.append(result)

        # Progress update
        if k % 100 == 0:  # Adjust frequency based on preference
            elapsed_time = time.time() - start_time
            print(f"Processed {k} alignments in {elapsed_time:.2f} seconds.")

# Display summary of significant results
print(f"\nTotal significant homologies found: {len(significant_homologies)}")
for homology in significant_homologies:
    print(f"Significant homology found:\n"
          f"Input Sequence ID: {homology['Input_Sequence_ID']}\n"
          f"Output Protein ID: {homology['Output_Protein_ID']}\n"
          f"Product: {homology['Output_Product']}\n"
          f"Similarity: {homology['Similarity']:.2f}\n"
          f"Alignment:\n{homology['Alignment']}\n")


Starting alignment of 371 protein sequences against 97435 output sequences.

Submitting tasks for input sequence 1/371

Submitting tasks for input sequence 2/371

Submitting tasks for input sequence 3/371

Submitting tasks for input sequence 4/371

Submitting tasks for input sequence 5/371

Submitting tasks for input sequence 6/371

Submitting tasks for input sequence 7/371

Submitting tasks for input sequence 8/371

Submitting tasks for input sequence 9/371

Submitting tasks for input sequence 10/371

Submitting tasks for input sequence 11/371

Submitting tasks for input sequence 12/371

Submitting tasks for input sequence 13/371

Submitting tasks for input sequence 14/371

Submitting tasks for input sequence 15/371

Submitting tasks for input sequence 16/371

Submitting tasks for input sequence 17/371

Submitting tasks for input sequence 18/371

Submitting tasks for input sequence 19/371

Submitting tasks for input sequence 20/371

Submitting tasks for input sequence 21/371

Submitti